# Task 5: SQL-based Analysis of Product Sales

In [1]:
import pandas as pd
import sqlite3

# Connect to the Chinook database
conn = sqlite3.connect("./Chinook.sqlite")

## 1. Top-Selling Products
We want to find the top 10 products (tracks) with the highest sales in terms of quantity sold.

In [2]:
query_top_selling_product = """
SELECT t.Name AS Track, SUM(ivl.Quantity) AS TotalSold
FROM InvoiceLine ivl
JOIN Track t ON ivl.TrackId = t.TrackId
GROUP BY t.Name
ORDER BY TotalSold DESC
LIMIT 10;
"""
df1 = pd.read_sql_query(query_top_selling_product, conn)
df1

,Track,TotalSold
0,The Trooper,5
1,Untitled,4
2,The Number Of The Beast,4
3,Sure Know Something,4
4,Hallowed Be Thy Name,4
5,Eruption,4
6,Where Eagles Dare,3
7,Welcome Home (Sanitarium),3
8,Sweetest Thing,3
9,Surrender,3


## 2. Revenue per Region
We calculate total revenue grouped by **billing country** (region).

In [3]:
query_revenue_per_region = """
SELECT BillingCountry AS Region, SUM(Total) AS TotalRevenue
FROM Invoice
GROUP BY BillingCountry
ORDER BY TotalRevenue DESC;
"""
df2 = pd.read_sql_query(query_revenue_per_region, conn)
df2

,Region,TotalRevenue
0,USA,523.06
1,Canada,303.96
2,France,195.10
3,Brazil,190.10
4,Germany,156.48
5,United Kingdom,112.86
6,Czech Republic,90.24
7,Portugal,77.24
8,India,75.26
9,Chile,46.62


## 3. Monthly Performance
We analyze sales performance by month to see trends over time.

In [4]:
query_monthly_performance = """
SELECT strftime('%Y-%m', InvoiceDate) AS Month, SUM(Total) AS MonthlyRevenue
FROM Invoice
GROUP BY Month
ORDER BY Month;
"""
df3 = pd.read_sql_query(query_monthly_performance, conn)
df3

,Month,MonthlyRevenue
0,2009-01,35.64
1,2009-02,37.62
2,2009-03,37.62
3,2009-04,37.62
4,2009-05,37.62
5,2009-06,37.62
6,2009-07,37.62
7,2009-08,37.62
8,2009-09,37.62
9,2009-10,37.62


## 4. Bonus: Ranking Products Using a Window Function
We use the SQL `RANK()` window function to rank products by total units sold.

In [5]:
query_rank = """
SELECT Track, TotalSold, RANK() OVER (ORDER BY TotalSold DESC) AS Rank
FROM (
    SELECT t.Name AS Track, SUM(ivl.Quantity) AS TotalSold
    FROM InvoiceLine ivl
    JOIN Track t ON ivl.TrackId = t.TrackId
    GROUP BY t.Name
);
"""
df4 = pd.read_sql_query(query_rank, conn)
df4

,Track,TotalSold,Rank
0,The Trooper,5,1
1,Eruption,4,2
2,Hallowed Be Thy Name,4,2
3,Sure Know Something,4,2
4,The Number Of The Beast,4,2
...,...,...,...
1883,Água E Fogo,1,321
1884,Água de Beber,1,321
1885,É Fogo,1,321
1886,"Étude 1, In C Major - Preludio (Presto) - Liszt",1,321


## Closing the Connection

In [6]:
conn.close()